# Substance Abuse Project

To be able to predict the outcome of substance abuse treatment is a high value solution. A model being able to make the distinction between a passing and failing case is difficult and requires alot of information. For this iteration, we utlized the TEDs 2018 dataset provided by the **_Center for Behavioral Health Statistics and Quality_** and the **_Substance Abuse and Mental Health Services Administration_**

---


## Installing Dependencies
If necessary please uncomment the following lines and install the dependencies with pip. Our notebook uses each one of these and it's important that all can be imported to avoid errors.

In [1]:
# !pip install sagemaker
# !pip install matplotlib
# !pip install boto3
# !pip install pandas
# !pip install fsspec
# !pip install s3fs

## Transforming Data
For this section we work on removing uninmportant data, unlabelled rows, and modifying our dataset to be used with the XGBoost training algorithm.
### Importing Dependencies and Setting Environment Variables
In this next code block we import boto3 and pandas. Pandas will be important for our first step, importing the CSV file as a **dataframe** a sort of programatically accessible table that we can use in our python code. boto3 will become important in a later step when we need to export our split data back to s3. Though there are other popular dataframe modules such as dask, pandas remains the most open source with the best source documentation and clear instruction. It also features the fastest operation time. We also import math, json, numpy and matplotlib.pyplot for later use. Below that we set our environment variables which we use throughout the notebook.

We also save our important filenames and the name of our bucket to the notebook for use later. Putting all this data together in one step means avoiding running extraneous code when rerunning code in later sequences

In [2]:
import boto3, pandas as pd, math, json, numpy as np, matplotlib.pyplot as plt, os

bucket = 'bucket-sagemaker-substance-abuse'
raw_file_2017 = 'data/raw/tedsd_puf_2017.csv'
raw_file_2018 = 'data/raw/tedsd_puf_2018.csv'
train_subfolder = 'data/train/'
eval_subfolder = 'data/evaluation/'
label = "REASON" # Desired Label
objective = "binary:logistic" # binary:logistic or multi:softmax
training_rounds = 10 # 1000-1500 for production, 10-100 for testing and development

### Saving our CSV as a pandas Dataframe Object
Most of the lines in this block format a string for use with pandas, telling it where our file is located. I've opted for *join concatenation with a slash* instead of operator concatenation which uses more memory and is less efficient. The latter half of this block of code reads out the data from our source file to the pandas dataframe which we can mutate during our transformations. Any dataframes method with a possible `inplace` attribute should be set to true so that the dataframe is mutated and not returned except for in the case that you're saving it to a variable. 

This step may take a few seconds

In [3]:
raw_location_2018 = 's3://{}/{}'.format(bucket, raw_file_2018)
raw_location_2017 = 's3://{}/{}'.format(bucket, raw_file_2017)
df_2018 = pd.read_csv(raw_location_2018)
print(f"2018 Dataframe Loaded: { len(df_2018) } rows x {len(df_2018.columns)} columns")
df_2017 = pd.read_csv(raw_location_2017)
print(f"2017 Dataframe Loaded: { len(df_2017) } rows x {len(df_2017.columns)} columns")
df = pd.concat([df_2018, df_2017])
print(f"Dataframe Loaded: { len(df) } rows x {len(df.columns)} columns")

2018 Dataframe Loaded: 1666366 rows x 76 columns
2017 Dataframe Loaded: 1661207 rows x 76 columns
Dataframe Loaded: 3327573 rows x 76 columns


### Removing Extraneous Columns
In the next step we take the names of all the columns we want to remove and set them in an array. Originally we would use an array to loop through and drop them one by one however pandas `drop` function can take *list-like* parameters and so we can actually save both time and resources by doing it all at once. This will significantly increase development speeds

In [4]:
remove = ["CASEID", "CBSA2010", "EMPLOY_D", "DETNLF_D", "LIVARAG_D", "ARRESTS_D", "SERVICES_D", "SUB1_D", "FREQ1_D", "SUB2_D", "FREQ2_D", "SUB3_D", "FREQ3_D", "FREQ_ATND_SELF_HELP_D", "DIVISION", "LOS", "DISYR"]
index_names = df[ (df['REASON'] == 3) | (df['REASON'] == 5) | (df['REASON'] == 6) | (df['REASON'] == 7)].index # Collects Death, Incarceration, Termination, and Other labeled rows for removal
df.drop(index_names, inplace = True) # Drops rows grabbed above
df.drop(remove, inplace=True, axis=1) # Drops extraneous columns in 'remove' array above
df.dropna(axis=0, subset=[label], inplace=True) # Drops rows missing desired label
print(f"Dataframe Transformed: { len(df) } rows x {len(df.columns)} columns")

Dataframe Transformed: 2571774 rows x 59 columns


### Transforming Categorical Data to Binary Columns
<span style="color:red">**BUG**</span> For this step we transform our categorical data into binary columns. To do this we call `get_dummies` on each column listed in the transform array as we loop through it. We then take that *dummy* dataframe and we concatenate it with the current dataframe along the vertical axis. After we loop through the entire array, we drop all the coulmns in the transform array as they're now redundant. Because there is a bug in this step, probably caused by a memory shortage or a CPU resource shortage, we should think about creating an empty dataframe in which we concatenate all dummies together and then concatenate that at the end instead of many times during the loop. In the last update I removed "Reason" from the transform list as we'll need to know what that becomes

In [5]:
transform = ["AGE", "GENDER", "RACE", "ETHNIC", "MARSTAT", "EDUC", "EMPLOY", "DETNLF", "PREG", "VET", "LIVARAG", "PRIMINC", "ARRESTS", "SERVICES", "METHUSE", "DAYWAIT", "PSOURCE", "DETCRIM", "NOPRIOR", "SUB1", "ROUTE1", "FREQ1", "FRSTUSE1", "SUB2", "ROUTE2", "FREQ2", "FRSTUSE2", "SUB3", "ROUTE3", "FREQ3", "FRSTUSE3", "IDU", "REGION", "STFIPS"];
df = pd.get_dummies(df, prefix=transform, columns=transform)
print(f"Dataframe Transformed: { len(df) } rows x {len(df.columns)} columns")

Dataframe Transformed: 2571774 rows x 306 columns


### Convert to Binary Classification and Modify Data for Algorithm
This code moves our label to the front of the dataframe. It also decrements 1 from the Reason category so the the XDGBoost Algorithm can understand the classes. Our current categories are:
- **0**: Positive Result
- **1**: Non-positive Result

For reasons having to do with confidence, we've phased out multi classification, though it can be reenabled by modifying the hyperparameters for the algorithm and modifying replace statements below. THe replace methods called may seem redundant but they are created in a way so that commenting out certain statements allows us to roll back to multi.

In [6]:
df[label] = df[label].replace([3, 4], [2, 2]) # Converts to Binary
column = df.pop(label)
df.insert(0, label, column)
df[label]-=1
class_pool = pd.get_dummies(df[label]) # For Debugging
print(f"Labels altered, class pool reduced to {len(class_pool.columns)}")

Labels altered, class pool reduced to 2


## Split Data
To do proper model training we'll use a holdout dataset for evaluation and so we'll need to split the set into 80% and 20%
### Separating Training and Evaluation Sets and Uploading them to S3
To complete our tranformation section, we grab a random sample of our data that makes up half of our data and we transport it to S3 utilizing a boto3 s3 client after converting the new dataframe to a CSV. If the command is run twice it will overwrite previous data. This section also creates an Evaluation Dataframe and ships it in a CSV to a seperate folder from the training data.

This process will take up to 3 minutes

In [7]:
sdf = df.sample(frac=1)
print(f"Random Dataframe Created: { len(sdf) } rows x {len(sdf.columns)} columns")
training_end_index = math.floor(len(sdf)*0.8)
eval_start_index = training_end_index + 1
end = len(sdf) - 1
tdf = sdf.iloc[0:training_end_index] # first five rows of dataframe
print(f"Training Dataframe Created: { len(tdf) } rows x {len(tdf.columns)} columns")
edf = sdf.iloc[training_end_index:end]
print(f"Evaluation Dataframe Created: { len(edf) } rows x {len(edf.columns)} columns")

t_filename = 'split-data/training_data.csv'
e_filename = 'split-data/evaluation_data.csv'
tdf.to_csv(t_filename, header=False, index=False)
edf.to_csv(e_filename, header=False, index=False)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(t_filename, bucket, train_subfolder + t_filename)
s3.meta.client.upload_file(e_filename, bucket, eval_subfolder + e_filename)
print(f"Operation Complete, Files Uploaded")



Random Dataframe Created: 2571774 rows x 306 columns
Training Dataframe Created: 2057419 rows x 306 columns
Evaluation Dataframe Created: 514354 rows x 306 columns
Operation Complete, Files Uploaded


In [8]:
print(f"deleting local files")
os.remove(t_filename)
os.remove(e_filename)
print(f"local files deleted")

deleting local files
local files deleted


## Training the Model

### Setting Sagemaker Environment
Now we get to important stuff. Here we leave the transformation phase and move into the training phase. This block saves our region into the notebook making it usable in our model. The steps below do not need to be run in conjunction with those above, provided you've already completed the above steps

In [9]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

AWS Region: us-east-2


### Setting Model Parameters and HyperParameters
In this section we set up our XDGBoost training and pass it our basic values and hyperparameters. `num_round` determines how many rounds of training the model goes through. `num_class` alters how many classes it accepts. If the objective changes to binary this can be removed. Using A/B Testing I was able to discover there's not really a point to doing more than *1000* rounds of training. The training m-error may keep dropping but the evaluation m-error stays roughly stagnant

In [10]:
from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput

model_bucket_prefix = 'output/models'

s3_output_location = 's3://{}/{}/{}'.format(bucket, model_bucket_prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role = 'arn:aws:iam::620700586481:role/sagemaker-substance-abuse-project-role',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

xgb_model.set_hyperparameters(
    max_depth = 6,
    eta = 0.3,
    min_child_weight = 1,
    subsample = 0.9,
    objective = objective,
    # num_class = 3,
    num_round = training_rounds 
)

257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.2-1


### Load Data from S3 as Training Inputs
Here we let SageMaker know what we intend to use for training and evaluationcand where it can find our CSV files in S3

In [11]:
from sagemaker.session import TrainingInput

prefix = "data"

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train/training_data.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "evaluation/evaluation_data.csv"), content_type="csv"
)

### Training the Model
This part begins the training and the evaluation in one step. We pass it both our training input and our validation input.

In [12]:
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

2021-08-12 18:59:42 Starting - Starting the training job...
2021-08-12 19:00:06 Starting - Launching requested ML instancesCreateXgboostReport: InProgress
ProfilerReport-1628794782: InProgress
...
2021-08-12 19:00:37 Starting - Preparing the instances for training.........
2021-08-12 19:02:12 Downloading - Downloading input data......
2021-08-12 19:03:11 Training - Downloading the training image...
2021-08-12 19:03:38 Training - Training image download completed. Training in progress.[2021-08-12 19:03:33.756 ip-10-0-193-51.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined d

### Debug and Visualize Output
Within this step, mostly for debugging and visualization purposes, we call to our output and confirm the files were created. Then we get the link to our html file which contains our confusion matrix and other data.

In [14]:
rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive
from IPython.display import FileLink, FileLinks
display("Click link below to view the XGBoost Training report", FileLink("CreateXgboostReport/xgboost_report.html"))

2021-08-12 19:07:36     617396 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/CreateXgboostReport/xgboost_report.html
2021-08-12 19:07:36     418220 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/CreateXgboostReport/xgboost_report.ipynb
2021-08-12 19:06:44     322348 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/ProfilerReport-1628794782/profiler-output/profiler-report.html
2021-08-12 19:06:44     168692 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/ProfilerReport-1628794782/profiler-output/profiler-report.ipynb
2021-08-12 19:06:41        191 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/ProfilerReport-1628794782/profiler-output/profiler-reports/BatchSize.json
2021-08-12 19:06:41        199 output/models/xgboost_model/sagemaker-xgboost-2021-08-12-18-59-42-514/rule-output/ProfilerReport-1628794782/profiler-outpu

'Click link below to view the XGBoost Training report'

/root/predicting-the-future-TEDS/CreateXgboostReport/xgboost_report.html

### Project Goals
- [x] Get data into sagemaker
- [x] Clean Data
- [x] Transform Data
- [x] Split Data
- [x] Train Model
- [ ] Modify Patterns and remove rows with now extraneous labels
- [ ] Reach 80-85% Accuracy over the next 3 days
- [ ] Deploy Model
- [ ] Create Demo?